# Ejercicio 10: Re-ranking

**Objetivo:** Implementar y evaluar un pipeline de Recuperación de Información en dos etapas, y analizar el impacto del re-ranking en la calidad del ranking.

## Parte 1. Preparación del corpus

* Cargar el corpus (documentos/pasajes).
* Cargar las consultas (queries).
* Cargar qrels (relevancia).

In [2]:
!pip install beir

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.8/304.8 kB 19.6 MB/s eta 0:00:00


In [3]:
from beir import util
from beir.datasets.data_loader import GenericDataLoader
import pandas as pd
from tqdm.autonotebook import tqdm

/usr/local/lib/python3.12/dist-packages/beir/util.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [4]:
DATASET_NAME = "scifact"
DATA_DIR = "../data/beir_datasets"
url = f"https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/{DATASET_NAME}.zip"
util.download_and_unzip(url, DATA_DIR)

../data/beir_datasets/scifact.zip:   0%|          | 0.00/2.69M [00:00<?, ?iB/s]

'../data/beir_datasets/scifact'

In [19]:
dataset_path = DATA_DIR + "/" + DATASET_NAME
corpus, queries, qrels = GenericDataLoader(dataset_path).load(split="test")

  0%|          | 0/5183 [00:00<?, ?it/s]

In [6]:
df_corpus = (
    pd.DataFrame.from_dict(corpus, orient="index")
      .reset_index()
      .rename(columns={"index": "doc_id"})
)

df_corpus

,doc_id,text,title
0,4983,Alterations of the architecture of cerebral wh...,Microstructural development of human newborn c...
1,5836,Myelodysplastic syndromes (MDS) are age-depend...,Induction of myelodysplasia by myeloid-derived...
2,7912,ID elements are short interspersed elements (S...,"BC1 RNA, the transcript from a master gene for..."
3,18670,DNA methylation plays an important role in bio...,The DNA Methylome of Human Peripheral Blood Mo...
4,19238,Two human Golli (for gene expressed in the oli...,The human myelin basic protein gene is include...
...,...,...,...
5178,195689316,BACKGROUND The main associations of body-mass ...,Body-mass index and cause-specific mortality i...
5179,195689757,A key aberrant biological difference between t...,Targeting metabolic remodeling in glioblastoma...
5180,196664003,A signaling pathway transmits information from...,Signaling architectures that transmit unidirec...
5181,198133135,AIMS Trabecular bone score (TBS) is a surrogat...,"Association between pre-diabetes, type 2 diabe..."


In [7]:
df_queries = (
    pd.DataFrame.from_dict(queries, orient="index", columns=["query"])
      .reset_index()
      .rename(columns={"index": "query_id"})
)

df_queries

,query_id,query
0,1,0-dimensional biomaterials show inductive prop...
1,3,"1,000 genomes project enables mapping of genet..."
2,5,1/2000 in UK have abnormal PrP positivity.
3,13,5% of perinatal mortality is due to low birth ...
4,36,A deficiency of vitamin B12 increases blood le...
...,...,...
295,1379,Women with a higher birth weight are more like...
296,1382,aPKCz causes tumour enhancement by affecting g...
297,1385,cSMAC formation enhances weak ligand signalling.
298,1389,mTORC2 regulates intracellular cysteine levels...


In [8]:
rows = []
for qid, docs in qrels.items():
    for doc_id, rel in docs.items():
        rows.append({
            "query_id": qid,
            "doc_id": doc_id,
            "relevance": rel
        })

df_qrels = pd.DataFrame(rows)
df_qrels

,query_id,doc_id,relevance
0,1,31715818,1
1,3,14717500,1
2,5,13734012,1
3,13,1606628,1
4,36,5152028,1
...,...,...,...
334,1379,17450673,1
335,1382,17755060,1
336,1385,306006,1
337,1389,23895668,1


In [9]:
# Elegimos una query cualquiera que tenga varios documentos relevantes
qid = "133"

print("Query:")
print(df_queries.loc[df_queries["query_id"] == qid, "query"].values[0])

print("\nDocumentos relevantes para esta query:")
df_qrels[(df_qrels["query_id"] == qid) & (df_qrels["relevance"] > 0)]

Query:
Assembly of invadopodia is triggered by focal generation of phosphatidylinositol-3,4-biphosphate and the activation of the nonreceptor tyrosine kinase Src.

Documentos relevantes para esta query:


,query_id,doc_id,relevance
31,133,38485364,1
32,133,6969753,1
33,133,17934082,1
34,133,16280642,1
35,133,12640810,1


## Parte 2. Retrieval inicial (baseline)

* Implementar retrieval inicial con BM25
* Obtener métricas: Recall@10 nDCG@10

In [11]:
import nltk

nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [12]:
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

class ModeloBM25:
    """
    Implementación del Modelo de Ranking BM25.
    Modificado para manejar IDs de documento originales (strings) para compatibilidad con BEIR.
    """
    def __init__(self, k1=1.2, b=0.75, idioma='spanish'):
        self.k1 = k1                       # Parámetro de ajuste de saturación de TF
        self.b = b                         # Parámetro de ajuste de normalización por longitud
        self.vocabulario = {}              # Término a ID (índice de columna)
        self.listaStopwords = set(stopwords.words(idioma))
        self.original_doc_ids = []         # Lista para almacenar los IDs de documento originales (strings)
        self.matrizFrecuencia = None       # Matriz de NumPy (Documentos x Términos)
        self.vectorLongitudDocumento = None# Vector con la longitud de cada documento |D|
        self.longitudPromedio = 0.0        # Longitud promedio de los documentos avgdl
        self.vectorIdf = None              # Vector de NumPy con los pesos IDF de BM25

    def preProcesar(self, texto):
        """ Tokenización y eliminación de stopwords. """
        textoMin = texto.lower()
        tokens = word_tokenize(textoMin)
        tokensFiltrados = [
            token for token in tokens
            if token.isalpha() and token not in self.listaStopwords
        ]
        return tokensFiltrados

    # --- Ajuste (Fit) del Modelo ---

    def ajustarCorpus(self, serieDocumentos):
        """
        Crea el vocabulario, calcula las longitudes de documento, IDF,
        y la matriz de frecuencia necesaria para la puntuación.
        `serieDocumentos` debe ser un diccionario o Series con doc_id -> texto.
        """
        documentosTokenizados = []
        longitudes = []

        # 1. Tokenizar, generar vocabulario y calcular longitudes
        for internal_idx, (original_doc_id, texto) in enumerate(serieDocumentos.items()):
            tokens = self.preProcesar(texto)
            documentosTokenizados.append(tokens)
            longitudes.append(len(tokens))
            self.original_doc_ids.append(original_doc_id)
            for token in tokens:
                if token not in self.vocabulario:
                    self.vocabulario[token] = len(self.vocabulario)

        self.vectorLongitudDocumento = np.array(longitudes, dtype=float)
        self.numDocumentos = len(self.original_doc_ids) # Use length of stored original IDs
        self.longitudPromedio = np.mean(self.vectorLongitudDocumento)
        numTerminos = len(self.vocabulario)

        # 2. Crear Matriz de Frecuencia de Término (Count Matrix)
        self.matrizFrecuencia = np.zeros((self.numDocumentos, numTerminos), dtype=np.int32)
        for docIndex, tokens in enumerate(documentosTokenizados):
            for token in tokens:
                if token in self.vocabulario:
                    terminoIndex = self.vocabulario[token]
                    self.matrizFrecuencia[docIndex, terminoIndex] += 1

        # 3. Calcular IDF (Específico de BM25)
        # BM25 IDF: log( (N - df_t + 0.5) / (df_t + 0.5) )
        documentosConTermino = np.sum(self.matrizFrecuencia > 0, axis=0) # df_t
        N = self.numDocumentos

        # Uso de NumPy para aplicar la fórmula a todos los términos
        self.vectorIdf = np.log((N - documentosConTermino + 0.5) / (documentosConTermino + 0.5))

        print(f"Ajuste completado. Documentos: {self.numDocumentos}, Términos: {numTerminos}")
        print(f"Longitud Promedio (avgdl): {self.longitudPromedio:.2f}")


    # --- Búsqueda (Search) del Modelo ---

    def buscar(self, consulta, k=3):
        """
        Calcula las puntuaciones BM25 para la consulta y ranquea los documentos.
        Devuelve los IDs de documento originales (strings) y las puntuaciones.
        """
        tokensConsulta = self.preProcesar(consulta)
        puntuaciones = np.zeros(self.numDocumentos, dtype=float)

        # Normalización por longitud (B): k1 * (1 - b + b * (|D| / avgdl))
        normalizacionDoc = self.k1 * (
            (1 - self.b) + self.b * (self.vectorLongitudDocumento / self.longitudPromedio)
        )

        for token in tokensConsulta:
            if token in self.vocabulario:
                terminoIndex = self.vocabulario[token]
                # Obtener la columna de IDF y la columna de frecuencia (tf)
                idfTermino = self.vectorIdf[terminoIndex]
                frecuenciasTermino = self.matrizFrecuencia[:, terminoIndex] # f(t_i, D)

                # Expresión del numerador de la fórmula: f(t_i, D) * (k1 + 1)
                numerador = frecuenciasTermino * (self.k1 + 1)

                # Expresión del denominador: f(t_i, D) + Normalización por longitud
                denominador = frecuenciasTermino + normalizacionDoc

                # Acumular puntuaciones (IDF * Factor de saturación/normalización)
                if idfTermino > 0:
                    puntuaciones += idfTermino * (numerador / denominador)

        # 1. Obtener los índices de los documentos ordenados por puntuación (descendente)
        indicesOrdenados = np.argsort(puntuaciones)[::-1]

        # 2. Seleccionar los top K documentos con puntuaciones > 0
        topKIndices = [i for i in indicesOrdenados if puntuaciones[i] > 0][:k]
        topKScores = puntuaciones[topKIndices]

        if len(topKIndices) == 0:
            print("No se encontraron documentos relevantes.")
            return []

        # Mapear los índices internos a los IDs de documento originales
        resultados = [
            (self.original_doc_ids[i], topKScores[idx])
            for idx, i in enumerate(topKIndices)
        ]

        print(f"Top {k} resultados encontrados (ID original, Puntuación BM25):")
        return resultados

Después de incluir nuestra implementación de BM25, inicializamos un objeto para ajustarlo a nuestro corpus.

In [20]:
bm25_model = ModeloBM25(idioma="english")

corpus_modified = {doc_id: doc_content["text"] for doc_id, doc_content in corpus.items()}

bm25_model.ajustarCorpus(corpus_modified)

Ajuste completado. Documentos: 5183, Términos: 27699
Longitud Promedio (avgdl): 116.03


Realizamos la búsqueda con las queries y las guardamos en forma de diccionario que la librería `bir` es capaz de interpretar.

In [21]:
dic_bm25_results = {}

for query_id, query_text in queries.items():
  results = bm25_model.buscar(query_text, k=100)
  dic_bm25_results[query_id] = {doc_id: float(score) for doc_id, score in results}

Top 100 resultados encontrados (ID original, Puntuación BM25):
Top 100 resultados encontrados (ID original, Puntuación BM25):
Top 100 resultados encontrados (ID original, Puntuación BM25):
Top 100 resultados encontrados (ID original, Puntuación BM25):
Top 100 resultados encontrados (ID original, Puntuación BM25):
Top 100 resultados encontrados (ID original, Puntuación BM25):
Top 100 resultados encontrados (ID original, Puntuación BM25):
Top 100 resultados encontrados (ID original, Puntuación BM25):
Top 100 resultados encontrados (ID original, Puntuación BM25):
Top 100 resultados encontrados (ID original, Puntuación BM25):
Top 100 resultados encontrados (ID original, Puntuación BM25):
Top 100 resultados encontrados (ID original, Puntuación BM25):
Top 100 resultados encontrados (ID original, Puntuación BM25):
Top 100 resultados encontrados (ID original, Puntuación BM25):
Top 100 resultados encontrados (ID original, Puntuación BM25):
Top 100 resultados encontrados (ID original, Puntuación

Evaluamos finalmente el modelo usando las métricas adoptadas.

In [22]:
from beir.retrieval.evaluation import EvaluateRetrieval

evaluator = EvaluateRetrieval(k_values=[1, 3, 5, 10, 100])

bm25_scores = evaluator.evaluate(qrels, dic_bm25_results, k_values=[1, 3, 5, 10, 100])

# Las métricas guardadas en la tupla
ndcg_scores = bm25_scores[0]
map_scores = bm25_scores[1]
recall_scores = bm25_scores[2]
precision_scores = bm25_scores[3]

print("\nMétricas de BM25 para k=10:")
print(f"NDCG@10: {ndcg_scores['NDCG@10']:.4f}")
print(f"MAP@10: {map_scores['MAP@10']:.4f}")
print(f"Recall@10: {recall_scores['Recall@10']:.4f}")
print(f"Precision@10: {precision_scores['P@10']:.4f}")

print("\nMétricas de BM25 para k=100:")
print(f"NDCG@100: {ndcg_scores['NDCG@100']:.4f}")
print(f"MAP@100: {map_scores['MAP@100']:.4f}")
print(f"Recall@100: {recall_scores['Recall@100']:.4f}")
print(f"Precision@100: {precision_scores['P@100']:.4f}")


Métricas de BM25 para k=10:
NDCG@10: 0.5567
MAP@10: 0.5150
Recall@10: 0.6713
Precision@10: 0.0737

Métricas de BM25 para k=100:
NDCG@100: 0.5861
MAP@100: 0.5215
Recall@100: 0.8026
Precision@100: 0.0091


## Parte 3. Implementación del re-ranking _cross-encoder_

* Re-rankear los top-k candidatos para cada query.
* Identificar qué documentos cambian de posición en el top 10

In [23]:
from beir.reranking.models import CrossEncoder
from beir.reranking import Rerank

cross_encoder_model = CrossEncoder('cross-encoder/ms-marco-electra-base')
reranker = Rerank(cross_encoder_model, batch_size=128)

Después de obtener el modelo, realizamos el reranking de los resultados de BM25 para k=100.

In [25]:
rerank_results = reranker.rerank(corpus, queries, dic_bm25_results, top_k=100)

Batches:   0%|          | 0/233 [00:00<?, ?it/s]

Finalmente, evaluamos el rendimiento del reranking utilizando las métricas anteriores.

In [26]:
cross_scores = evaluator.evaluate(qrels, rerank_results, k_values=[1, 3, 5, 10, 100])

# Las métricas guardadas en la tupla
ndcg_scores = bm25_scores[0]
map_scores = bm25_scores[1]
recall_scores = bm25_scores[2]
precision_scores = bm25_scores[3]

print("\nMétricas de BM25 para k=10:")
print(f"NDCG@10: {ndcg_scores['NDCG@10']:.4f}")
print(f"MAP@10: {map_scores['MAP@10']:.4f}")
print(f"Recall@10: {recall_scores['Recall@10']:.4f}")
print(f"Precision@10: {precision_scores['P@10']:.4f}")

print("\nMétricas de BM25 para k=100:")
print(f"NDCG@100: {ndcg_scores['NDCG@100']:.4f}")
print(f"MAP@100: {map_scores['MAP@100']:.4f}")
print(f"Recall@100: {recall_scores['Recall@100']:.4f}")
print(f"Precision@100: {precision_scores['P@100']:.4f}")


Métricas de BM25 para k=10:
NDCG@10: 0.5567
MAP@10: 0.5150
Recall@10: 0.6713
Precision@10: 0.0737

Métricas de BM25 para k=100:
NDCG@100: 0.5861
MAP@100: 0.5215
Recall@100: 0.8026
Precision@100: 0.0091


Finalmente, comprobamos si la posición de los documentos en el top 10 de cada query ha cambiado.

In [27]:
def get_top_10_doc_ids(results_dict, query_id):
  if query_id not in results_dict:
      return []
  # Ordenar por puntuación descendente
  sorted_docs = sorted(results_dict[query_id].items(), key=lambda item: item[1], reverse=True)
  return [doc_id for doc_id, _ in sorted_docs[:10]]

In [29]:
import collections
comparison_details = collections.defaultdict(dict)

print("Cambios en las posiciones en el top 10...")
for query_id in queries.keys():
  bm25_top10 = get_top_10_doc_ids(dic_bm25_results, query_id)
  rerank_top10 = get_top_10_doc_ids(rerank_results, query_id)

  # Documentos en el top 10 de BM25 pero no en el top 10 de reranked
  bm25_only = [doc_id for doc_id in bm25_top10 if doc_id not in rerank_top10]

  # Documentos en el top 10 de reranked pero no en el top 10 de BM25
  rerank_only = [doc_id for doc_id in rerank_top10 if doc_id not in bm25_top10]

  # Documentos comunes y cambios
  common_docs = set(bm25_top10).intersection(set(rerank_top10))
  rank_changes = []
  for doc_id in common_docs:
    bm25_rank = bm25_top10.index(doc_id) + 1
    rerank_rank = rerank_top10.index(doc_id) + 1
    if bm25_rank != rerank_rank:
      rank_changes.append({'doc_id': doc_id, 'bm25_rank': bm25_rank, 'rerank_rank': rerank_rank})

  comparison_details[query_id] = {
    'bm25_top10': bm25_top10,
    'rerank_top10': rerank_top10,
    'bm25_only': bm25_only,
    'rerank_only': rerank_only,
    'rank_changes': rank_changes
  }

Cambios en las posiciones en el top 10...


In [31]:
print("\n--- Resumen del Impacto del Re-ranking en los 10 Mejores Resultados ---")

total_queries = len(queries)
total_bm25_only_docs = 0
total_rerank_only_docs = 0
total_rank_changes = 0
queries_with_changes = 0

significant_change_examples = []

for query_id, details in comparison_details.items():
    bm25_only_count = len(details['bm25_only'])
    rerank_only_count = len(details['rerank_only'])
    rank_changes_count = len(details['rank_changes'])

    if bm25_only_count > 0 or rerank_only_count > 0 or rank_changes_count > 0:
        queries_with_changes += 1

    total_bm25_only_docs += bm25_only_count
    total_rerank_only_docs += rerank_only_count
    total_rank_changes += rank_changes_count

    if (bm25_only_count + rerank_only_count + rank_changes_count) > 3 and len(significant_change_examples) < 3:
        significant_change_examples.append(query_id)

print(f"Total de Consultas Procesadas: {total_queries}")
print(f"Consultas con algón cambio en el Top 10 después del re-ranking: {queries_with_changes} ({queries_with_changes/total_queries:.2%})")
print(f"Documentos totales eliminados del Top 10 de BM25 por el re-ranker (solo en BM25): {total_bm25_only_docs}")
print(f"Documentos totales añadidos al Top 10 re-ranqueado (solo en re-rank): {total_rerank_only_docs}")
print(f"Documentos totales con cambios de posición dentro del Top 10: {total_rank_changes}")

print("\n--- Ejemplos de Consultas con Cambios Significativos en el Top 10 ---")
if significant_change_examples:
    for qid in significant_change_examples:
        print(f"\nID de Consulta: {qid}")
        print(f"  Texto de la Consulta: {queries[qid]}")
        print(f"  Documentos en el Top 10 de BM25 pero no en el Top 10 re-ranqueado: {comparison_details[qid]['bm25_only']}")
        print(f"  Documentos en el Top 10 re-ranqueado pero no en el Top 10 de BM25: {comparison_details[qid]['rerank_only']}")
        print(f"  Documentos con cambios de posición: {comparison_details[qid]['rank_changes']}")
else:
    print("No se encontraron consultas con cambios significativos segón los criterios.")


--- Resumen del Impacto del Re-ranking en los 10 Mejores Resultados ---
Total de Consultas Procesadas: 300
Consultas con algón cambio en el Top 10 después del re-ranking: 300 (100.00%)
Documentos totales eliminados del Top 10 de BM25 por el re-ranker (solo en BM25): 1832
Documentos totales añadidos al Top 10 re-ranqueado (solo en re-rank): 1832
Documentos totales con cambios de posición dentro del Top 10: 878

--- Ejemplos de Consultas con Cambios Significativos en el Top 10 ---

ID de Consulta: 1
  Texto de la Consulta: 0-dimensional biomaterials show inductive properties.
  Documentos en el Top 10 de BM25 pero no en el Top 10 re-ranqueado: ['24998637', '13231899', '12824568', '825728', '17518195', '42240424', '19651306', '4702639']
  Documentos en el Top 10 re-ranqueado pero no en el Top 10 de BM25: ['23244529', '25328476', '40769868', '36637129', '1156322', '43385013', '515489', '393001']
  Documentos con cambios de posición: [{'doc_id': '10931595', 'bm25_rank': 1, 'rerank_rank': 6

## Parte 4. Implementación del re-ranking _LTR_

* Re-rankear los top-k candidatos para cada query.
* Identificar qué documentos cambian de posición en el top 10

In [32]:
!pip install xgboost

In [34]:
import xgboost as xgb
import numpy as np

def extract_features(query_id, doc_id):
    bm25_score = dic_bm25_results.get(query_id, {}).get(doc_id, 0.0)
    # Solo el score
    return [bm25_score]

X_train = []
y_train = []
groups = []

for qid, rel_docs in list(qrels.items()):
  count = 0
  for doc_id, score in rel_docs.items():
    X_train.append(extract_features(qid, doc_id))
    y_train.append(score)
    count += 1
  groups.append(count) # grupos para identificarlos

X_train = np.array(X_train)
y_train = np.array(y_train)

# Usamos el objetivo 'rank:ndcg' para optimizar la métrica NDCG
model = xgb.XGBRanker(
  objective='rank:ndcg',
  learning_rate=0.1,
  n_estimators=100,
  lambdarank_pair_method='topk' # Optimización tipo Pairwise
)

model.fit(X_train, y_train, group=groups)
print("Modelo XGBRanker entrenado utilizando scores BM25 como características.")

Modelo XGBRanker entrenado utilizando scores BM25 como características.


Ahora realizamos predicciones con el modelo para organizar el top 100 de resultados de recuperación para las queries.

In [43]:
ltr_rerank_results = {}

for query_id, bm25_docs in tqdm(dic_bm25_results.items(), desc="Predicting LTR scores"):
  # Descendiente
  sorted_bm25_docs = sorted(bm25_docs.items(), key=lambda item: item[1], reverse=True)[:100]

  query_doc_pairs = []
  features_for_prediction = []

  for doc_id, bm25_score in sorted_bm25_docs:
    # Extract features for prediction (only BM25 score in this case)
    # The extract_features function was already defined and handles this logic
    features = extract_features(query_id, doc_id)
    features_for_prediction.append(features)
    query_doc_pairs.append((doc_id, bm25_score))

  if features_for_prediction:
    # Convert features to a NumPy array for prediction
    features_for_prediction = np.array(features_for_prediction)

    # Predict LTR scores using the trained XGBRanker model
    ltr_scores = model.predict(features_for_prediction)

    # Store the predicted LTR scores in the desired format
    ltr_rerank_results[query_id] = {}
    for i, (doc_id, _) in enumerate(query_doc_pairs):
      ltr_rerank_results[query_id][doc_id] = float(ltr_scores[i])

Predicting LTR scores:   0%|          | 0/300 [00:00<?, ?it/s]

In [39]:
ltr_scores_evaluation = evaluator.evaluate(qrels, ltr_rerank_results, k_values=[1, 3, 5, 10, 100])

ndcg_ltr_scores = ltr_scores_evaluation[0]
map_ltr_scores = ltr_scores_evaluation[1]
recall_ltr_scores = ltr_scores_evaluation[2]
precision_ltr_scores = ltr_scores_evaluation[3]

print("\nMétricas de LTR Re-ranking para k=10:")
print(f"NDCG@10: {ndcg_ltr_scores['NDCG@10']:.4f}")
print(f"MAP@10: {map_ltr_scores['MAP@10']:.4f}")
print(f"Recall@10: {recall_ltr_scores['Recall@10']:.4f}")
print(f"Precision@10: {precision_ltr_scores['P@10']:.4f}")

print("\nMétricas de LTR Re-ranking para k=100:")
print(f"NDCG@100: {ndcg_ltr_scores['NDCG@100']:.4f}")
print(f"MAP@100: {map_ltr_scores['MAP@100']:.4f}")
print(f"Recall@100: {recall_ltr_scores['Recall@100']:.4f}")
print(f"Precision@100: {precision_ltr_scores['P@100']:.4f}")


Métricas de LTR Re-ranking para k=10:
NDCG@10: 0.0406
MAP@10: 0.0294
Recall@10: 0.0725
Precision@10: 0.0083

Métricas de LTR Re-ranking para k=100:
NDCG@100: 0.1738
MAP@100: 0.0471
Recall@100: 0.8026
Precision@100: 0.0091


In [42]:
ltr_comparison_details = collections.defaultdict(dict)

print("Cambios en las posiciones en el top 10...")
for query_id in queries.keys():
  bm25_top10 = get_top_10_doc_ids(dic_bm25_results, query_id)
  ltr_top10 = get_top_10_doc_ids(ltr_rerank_results, query_id)

  # Documentos en el top 10 de BM25 pero no en el top 10 de LTR
  bm25_only = [doc_id for doc_id in bm25_top10 if doc_id not in ltr_top10]

  # Documentos en el top 10 de LTR pero no en el top 10 de BM25
  ltr_only = [doc_id for doc_id in ltr_top10 if doc_id not in bm25_top10]

  # Documentos comunes
  common_docs = set(bm25_top10).intersection(set(ltr_top10))
  rank_changes = []
  for doc_id in common_docs:
    bm25_rank = bm25_top10.index(doc_id) + 1
    ltr_rank = ltr_top10.index(doc_id) + 1
    if bm25_rank != ltr_rank:
      rank_changes.append({'doc_id': doc_id, 'bm25_rank': bm25_rank, 'ltr_rank': ltr_rank})

  ltr_comparison_details[query_id] = {
    'bm25_top10': bm25_top10,
    'ltr_top10': ltr_top10,
    'bm25_only': bm25_only,
    'ltr_only': ltr_only,
    'rank_changes': rank_changes
  }

Cambios en las posiciones en el top 10...


In [38]:
print("\n--- Resumen del Impacto del Re-ranking LTR en los 10 Mejores Resultados ---")

total_queries = len(queries)
total_bm25_only_docs_ltr = 0
total_ltr_only_docs = 0
total_ltr_rank_changes = 0
queries_with_ltr_changes = 0

significant_ltr_change_examples = []

for query_id, details in ltr_comparison_details.items():
    bm25_only_count = len(details['bm25_only'])
    ltr_only_count = len(details['ltr_only'])
    rank_changes_count = len(details['rank_changes'])

    if bm25_only_count > 0 or ltr_only_count > 0 or rank_changes_count > 0:
        queries_with_ltr_changes += 1

    total_bm25_only_docs_ltr += bm25_only_count
    total_ltr_only_docs += ltr_only_count
    total_ltr_rank_changes += rank_changes_count

    if (bm25_only_count + ltr_only_count + rank_changes_count) > 3 and len(significant_ltr_change_examples) < 3:
        significant_ltr_change_examples.append(query_id)

print(f"Total de Consultas Procesadas: {total_queries}")
print(f"Consultas con algún cambio en el Top 10 después del re-ranking LTR: {queries_with_ltr_changes} ({queries_with_ltr_changes/total_queries:.2%})")
print(f"Documentos totales eliminados del Top 10 de BM25 por el re-ranker LTR (solo en BM25): {total_bm25_only_docs_ltr}")
print(f"Documentos totales añadidos al Top 10 re-ranqueado LTR (solo en LTR): {total_ltr_only_docs}")
print(f"Documentos totales con cambios de posición dentro del Top 10 (LTR): {total_ltr_rank_changes}")

print("\n--- Ejemplos de Consultas con Cambios Significativos en el Top 10 (LTR) ---")
if significant_ltr_change_examples:
    for qid in significant_ltr_change_examples:
        print(f"\nID de Consulta: {qid}")
        print(f"  Texto de la Consulta: {queries[qid]}")
        print(f"  Documentos en el Top 10 de BM25 pero no en el Top 10 re-ranqueado LTR: {ltr_comparison_details[qid]['bm25_only']}")
        print(f"  Documentos en el Top 10 re-ranqueado LTR pero no en el Top 10 de BM25: {ltr_comparison_details[qid]['ltr_only']}")
        print(f"  Documentos con cambios de posición: {ltr_comparison_details[qid]['rank_changes']}")
else:
    print("No se encontraron consultas con cambios significativos según los criterios.")


--- Resumen del Impacto del Re-ranking LTR en los 10 Mejores Resultados ---
Total de Consultas Procesadas: 300
Consultas con algún cambio en el Top 10 después del re-ranking LTR: 0 (0.00%)
Documentos totales eliminados del Top 10 de BM25 por el re-ranker LTR (solo en BM25): 0
Documentos totales añadidos al Top 10 re-ranqueado LTR (solo en LTR): 0
Documentos totales con cambios de posición dentro del Top 10 (LTR): 0

--- Ejemplos de Consultas con Cambios Significativos en el Top 10 (LTR) ---
No se encontraron consultas con cambios significativos según los criterios.


## Parte 5. Evaluación post re-ranking

Calcular métricas:
* nDCG@10
* MAP
* Recall@10

BM25


In [44]:
bm25_scores = evaluator.evaluate(qrels, dic_bm25_results, k_values=[1, 3, 5, 10, 100])

# Las métricas guardadas en la tupla
ndcg_scores = bm25_scores[0]
map_scores = bm25_scores[1]
recall_scores = bm25_scores[2]
precision_scores = bm25_scores[3]

print("\nMétricas de BM25 para k=10:")
print(f"NDCG@10: {ndcg_scores['NDCG@10']:.4f}")
print(f"MAP@10: {map_scores['MAP@10']:.4f}")
print(f"Recall@10: {recall_scores['Recall@10']:.4f}")
print(f"Precision@10: {precision_scores['P@10']:.4f}")


Métricas de BM25 para k=10:
NDCG@10: 0.5567
MAP@10: 0.5150
Recall@10: 0.6713
Precision@10: 0.0737


Cross Encoder

In [45]:
cross_scores = evaluator.evaluate(qrels, rerank_results, k_values=[1, 3, 5, 10, 100])

# Las métricas guardadas en la tupla
ndcg_scores = bm25_scores[0]
map_scores = bm25_scores[1]
recall_scores = bm25_scores[2]
precision_scores = bm25_scores[3]

print("\nMétricas de BM25 para k=10:")
print(f"NDCG@10: {ndcg_scores['NDCG@10']:.4f}")
print(f"MAP@10: {map_scores['MAP@10']:.4f}")
print(f"Recall@10: {recall_scores['Recall@10']:.4f}")
print(f"Precision@10: {precision_scores['P@10']:.4f}")


Métricas de BM25 para k=10:
NDCG@10: 0.5567
MAP@10: 0.5150
Recall@10: 0.6713
Precision@10: 0.0737


LTR

In [46]:
ltr_scores_evaluation = evaluator.evaluate(qrels, ltr_rerank_results, k_values=[1, 3, 5, 10, 100])

# Las métricas guardadas en la tupla
ndcg_scores = ltr_scores_evaluation[0]
map_scores = ltr_scores_evaluation[1]
recall_scores = ltr_scores_evaluation[2]
precision_scores = ltr_scores_evaluation[3]

print("\nMétricas de LTR Re-ranking para k=10:")
print(f"NDCG@10: {ndcg_scores['NDCG@10']:.4f}")
print(f"MAP@10: {map_scores['MAP@10']:.4f}")
print(f"Recall@10: {recall_scores['Recall@10']:.4f}")
print(f"Precision@10: {precision_scores['P@10']:.4f}")


Métricas de LTR Re-ranking para k=10:
NDCG@10: 0.0406
MAP@10: 0.0294
Recall@10: 0.0725
Precision@10: 0.0083
